In [1]:
import os
import pandas as pd

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
model_path = "/home/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/"
file_path = "/home/xhl/eval/my_eval/data/math500/test.jsonl"
df = pd.read_json(file_path, lines=True)
problem_list = df["problem"][:4]

texts_fast = []
texts_slow = []

for prob in problem_list:
    texts_fast.append("<|User|>Return your final response within \\boxed{}."+prob+"\n<|Assistant|><think>\nTo")

for prob in problem_list:
    texts_slow.append("<|User|>Return your final response within \\boxed{}."+prob+"\n<|Assistant|><think>\nAlright")

In [2]:
from vllm import LLM, SamplingParams
llm = LLM(
    model=model_path,
    enable_steer_vector=True,
    enforce_eager=True,
    tensor_parallel_size=1,
    enable_prefix_caching=False,
    enable_chunked_prefill=False
)

INFO 01-11 21:42:57 [utils.py:253] non-default args: {'enable_prefix_caching': False, 'disable_log_stats': True, 'enforce_eager': True, 'enable_steer_vector': True, 'enable_chunked_prefill': False, 'model': '/home/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/'}
INFO 01-11 21:42:57 [model.py:514] Resolved architecture: Qwen2ForCausalLM
INFO 01-11 21:42:57 [model.py:1661] Using max model len 131072
WARNING 01-11 21:42:57 [arg_utils.py:1900] This model does not officially support disabling chunked prefill. Disabling this manually may cause the engine to crash or produce incorrect outputs.
WARNING 01-11 21:42:59 [vllm.py:625] Enforce eager set, overriding optimization level to -O0
INFO 01-11 21:42:59 [vllm.py:725] Cudagraph is disabled under eager mode
(EngineCore_DP0 pid=127452) INFO 01-11 21:42:59 [core.py:94] Initializing a V1 LLM engine (v0.1.dev12297+gb744aa686) with config: model='/home/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/', speculative_config=None,

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


(EngineCore_DP0 pid=127452) INFO 01-11 21:43:04 [default_loader.py:308] Loading weights took 1.00 seconds
(EngineCore_DP0 pid=127452) INFO 01-11 21:43:04 [steer_vector_model_runner_mixin.py:36] Initialized SteerVector worker manager
(EngineCore_DP0 pid=127452) INFO 01-11 21:43:04 [steer_vector_model_runner_mixin.py:50] Wrapping model with steer vector support
(EngineCore_DP0 pid=127452) WARNING 01-11 21:43:04 [models.py:483] No moe_layer modules found for steer vector intervention
(EngineCore_DP0 pid=127452) INFO 01-11 21:43:04 [capture_model_runner_mixin.py:113] [Capture] Wrapped 28 decoder layers for hidden states capture
(EngineCore_DP0 pid=127452) INFO 01-11 21:43:05 [gpu_model_runner.py:3731] Model loading took 3.3466 GiB memory and 1.378418 seconds
(EngineCore_DP0 pid=127452) INFO 01-11 21:43:09 [gpu_worker.py:376] Available KV cache memory: 31.21 GiB
(EngineCore_DP0 pid=127452) INFO 01-11 21:43:10 [kv_cache_utils.py:1309] GPU KV cache size: 1,168,752 tokens
(EngineCore_DP0 pid=1

In [3]:
answers_fast = llm.generate(
    texts_fast,
    SamplingParams(
        temperature=0,
        max_tokens=4096,
        skip_special_tokens=False,
    ),
)
answers_fast = [answer.outputs[0].text for answer in answers_fast]
qa_pairs_fast = [texts_fast[i] + answers_fast[i] for i in range(len(texts_fast))]

answers_slow = llm.generate(
    texts_slow,
    SamplingParams(
        temperature=0,
        max_tokens=4096,
        skip_special_tokens=False,
    ),
)
answers_slow = [answer.outputs[0].text for answer in answers_slow]
qa_pairs_slow = [texts_slow[i] + answers_slow[i] for i in range(len(texts_slow))]

Adding requests:   0%|          | 0/4 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/4 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, outpu

Adding requests:   0%|          | 0/4 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/4 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, outpu

In [4]:
import easysteer.hidden_states as hs
all_hidden_states, _= hs.get_all_hidden_states_generate(llm, qa_pairs_fast + qa_pairs_slow)

Adding requests:   0%|          | 0/8 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/8 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, outpu

In [5]:
from easysteer.steer import extract_pca_control_vector
control_vector = extract_pca_control_vector(
    all_hidden_states=all_hidden_states,
    positive_indices=list(range(4)), 
    negative_indices=list(range(4,8)),
    model_type="qwen2.5",
    method="center",
    token_pos=-1,
    normalize=False
)
control_vector.export_gguf("MATH500.gguf")

Computing PCA directions:   0%|          | 0/28 [00:00<?, ?it/s]

[2026-01-11 21:44:42] INFO pca.py:114: Layer 0: PCA on centered data explains 74.16408% of the variance
[2026-01-11 21:44:42] INFO pca.py:169: Layer 0: Direction corrected (flipped)
[2026-01-11 21:44:42] INFO pca.py:114: Layer 1: PCA on centered data explains 73.90079% of the variance
[2026-01-11 21:44:42] INFO pca.py:169: Layer 1: Direction corrected (flipped)
[2026-01-11 21:44:42] INFO pca.py:114: Layer 2: PCA on centered data explains 71.15234% of the variance
[2026-01-11 21:44:42] INFO pca.py:169: Layer 2: Direction corrected (flipped)
[2026-01-11 21:44:42] INFO pca.py:114: Layer 3: PCA on centered data explains 74.56971% of the variance
[2026-01-11 21:44:42] INFO pca.py:169: Layer 3: Direction corrected (flipped)
[2026-01-11 21:44:42] INFO pca.py:114: Layer 4: PCA on centered data explains 72.49410% of the variance
[2026-01-11 21:44:42] INFO pca.py:114: Layer 5: PCA on centered data explains 72.07904% of the variance
[2026-01-11 21:44:42] INFO pca.py:114: Layer 6: PCA on centered 